In [1]:
!pip install neo4j
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.6 MB/s eta 0:00:00:00:01


In [2]:
from neo4j import GraphDatabase

In [3]:
# Set up your credentials and URI for Neo4j Aura
uri = "YOUR_URI"
user = "neo4j" # It's bydefault neo4j
password = "YOUR_PASSWORD"

In [4]:
# Create a driver instance to connect to the database
driver = GraphDatabase.driver(uri, auth=(user, password))

In [5]:
# Function to create a person node
def create_person(tx, name, birth_year):
    tx.run("CREATE (p:Person {name: $name, birth_year: $birth_year})", name=name, birth_year=birth_year)

In [7]:
# Function to read persons
def read_persons(tx):
    result = tx.run("MATCH (p:Person) RETURN p.name AS name, p.birth_year AS birth_year")
    
    for record in result:
        print(f"{record['name']} (Born {record['birth_year']})")

In [10]:
# Create and query data in Neo4j
with driver.session() as session:
    # Write transaction: creating a new Person node
    session.write_transaction(create_person, "Barack Obama", 1961)
    session.write_transaction(create_person, "Elon Musk", 1971)

    # Read transaction: fetching all Person nodes
    session.read_transaction(read_persons)
    
    # Close the driver when done
    driver.close()

Barack Obama (Born 1961)
Elon Musk (Born 1971)
Barack Obama (Born 1961)
Elon Musk (Born 1971)


/tmp/ipykernel_36/1816081342.py:4: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_person, "Barack Obama", 1961)
/tmp/ipykernel_36/1816081342.py:5: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_person, "Elon Musk", 1971)
/tmp/ipykernel_36/1816081342.py:8: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(read_persons)


In [12]:
# Example of querying the knowledge graph using Cypher
def get_person_by_name(tx, name):
    result = tx.run("MATCH (p:Person {name: $name}) RETURN p.name, p.birth_year", name=name)
    
    for record in result:
        print(record["p.name"], record["p.birth_year"])

In [13]:
with driver.session() as session:
    session.read_transaction(get_person_by_name, "Barack Obama")

/tmp/ipykernel_36/768213969.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_36/768213969.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(get_person_by_name, "Barack Obama")


Barack Obama 1961
Barack Obama 1961


# Another example

### 1. Build KG with Neo4j

In [15]:
# Set up your credentials and URI for Neo4j Aura
uri = "neo4j+s://acf434a9.databases.neo4j.io"
user = "neo4j" # It's bydefault neo4j
password = "n5qg7KAqvSSTW2szWeZCLsU7xA0HuCWBD7nlnPJu_2o"

# Create a driver instance to connect to the database
driver = GraphDatabase.driver(uri, auth=(user, password))

In [16]:
# Create entities and relationships in the knowledge graph
def create_knowledge_graph(tx): 
    tx.run("CREATE (:Person {name: 'Elon Musk'})-[:CEO_OF]->(:Company {name: 'Tesla'})") 
    tx.run("CREATE (:Person {name: 'Satya Nadella'})-[:CEO_OF]->(:Company {name: 'Microsoft'})") 
    tx.run("CREATE (:Person {name: 'Sundar Pichai'})-[:CEO_OF]->(:Company {name: 'Google'})")

In [17]:
# Run the transaction to create the graph
with driver.session() as session: 
    session.write_transaction(create_knowledge_graph)
    driver.close()

/tmp/ipykernel_36/3565496405.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_knowledge_graph)


### 2. Query

In [18]:
def get_ceo_of_company(tx, company): 
    query = "MATCH (p:Person)-[:CEO_OF]->(c:Company {name: $company}) RETURN p.name AS CEO" 
    result = tx.run(query, company=company) 
    return result.single()['CEO']

In [19]:
# Example query to retrieve the CEO of Tesla
with driver.session() as session: 
    ceo = session.read_transaction(get_ceo_of_company, "Tesla") 
    print(f"The CEO of Tesla is {ceo}")

The CEO of Tesla is Elon Musk


/tmp/ipykernel_36/944754353.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_36/944754353.py:3: DeprecationWarning: read_transaction has been renamed to execute_read
  ceo = session.read_transaction(get_ceo_of_company, "Tesla")


### 3. Feed into GPT-2 for generation

In [20]:
from transformers import pipeline
# Load GPT-2 text generation model
generator = pipeline('text-generation', model='gpt2')

2025-07-29 12:45:05.832094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753793106.080131      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753793106.151791      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [22]:
# Combine the query with retrieved information from the knowledge graph
query = "Who is the CEO of Tesla?"
retrieved_info = f"The CEO of Tesla is {ceo}."

In [23]:
# Generate a detailed response using GPT-2
input_text = f"Query: {query}\nRelevant Information: {retrieved_info}"
response = generator(input_text, max_length=100, num_return_sequences=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [24]:
print("\nGenerated Response:")
print(response[0]['generated_text'])


Generated Response:
Query: Who is the CEO of Tesla?
Relevant Information: The CEO of Tesla is Elon Musk.
Tesla's current CEO, Elon Musk, is an investor and partner with the company's technology firm, Qantas
Tesla has a history of being a proponent of investing in the semiconductor industry. Musk worked in the semiconductor industry at Qualcomm, where he founded the company's technology division. Tesla was also involved in the acquisition of Intel over the years and its semiconductor division now manages its production of Intel chips.
Relevant Information: The Tesla CEO is Elon Musk.
Tesla's current CEO, Elon Musk, is an investor and partner with the company's technology firm, Qantas.
Tesla has a history of being a proponent of investing in the semiconductor industry. Musk worked in the semiconductor industry at Qualcomm, where he founded the company's technology division. Musk was also involved in the acquisition of Intel over the years and its semiconductor division now manages its p